<a href="https://colab.research.google.com/github/ukeshbhaal/marlo-assignment/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask
!pip install pymongo


from flask import Flask, request, jsonify
from pymongo import MongoClient

app = Flask(__name__)
client = MongoClient('mongodb+srv://ukesh:<ukesh>@cluster0.uo8lxtk.mongodb.net/')
db = client['ecom_db']

@app.route('/register', methods=['POST'])
def register():
    user_data = request.get_json()
    first_name = user_data['first_name']
    last_name = user_data['last_name']
    email = user_data['email']
    password = user_data['password']
    
    user = {'first_name': first_name, 'last_name': last_name, 'email': email, 'password': password}
    db.users.insert_one(user)
    
    return jsonify({'message': 'User registered successfully'})
@app.route('/login', methods=['POST'])
def login():
    login_data = request.get_json()
    email = login_data['email']
    password = login_data['password']
    
    user = db.users.find_one({'email': email})
    
    if user and user['password'] == password:
        return jsonify({'message': 'Login successful'})
    else:
        return jsonify({'message': 'Invalid credentials'})

@app.route('/upload', methods=['POST'])
def upload():
    if 'file' not in request.files:
        return jsonify({'message': 'No file selected'})
    
    file = request.files['file']
    
    if file.filename == '':
        return jsonify({'message': 'No file selected'})
    
    if file:
        products = []
        for line in file:
            data = line.decode('utf-8').strip().split(',')
            product = {'name': data[0], 'barcode': data[1], 'brand': data[2], 'description': data[3], 'price': float(data[4]), 'available': data[5].lower() == 'true'}
            products.append(product)
        
        db.products.insert_many(products)
        
        return jsonify({'message': 'File uploaded successfully'})
@app.route('/review', methods=['POST'])
def review():
    review_data = request.get_json()
    product_id = review_data['product_id']
    user_id = review_data['user_id']
    rating = review_data['rating']
    comment = review_data['comment']
    
    review = {'product_id': product_id, 'user_id': user_id, 'rating': rating, 'comment': comment}
    db.reviews.insert_one(review)
    
    return jsonify({'message': 'Review added successfully'})

@app.route('/products', methods=['GET'])
def products():
    page = int(request.args.get('page', 1))
    limit = int(request.args.get('limit', 10))
    sort = request.args.get('sort', 'rating')
    
    skip = (page - 1) * limit
    
    products = db.products.find().sort(sort, -1).skip(skip).limit(limit)
    
    return jsonify(list(products))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
